## Team Data Scraping

Different sources require different scraping techniques. 

The easiest (yet unfortunately less-desireable) source is basketball-reference.com. 
This site is html, and we can easily scrape anything from it using pd.read_html(url). 

The most difficult (and most desirable) source is stats.nba.com. This website uses no HTML and is completely dynamic, so I have to use selenium to scrape those statistics. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import warnings
warnings.filterwarnings('ignore')

from selenium.common.exceptions import WebDriverException


### Functions

In [2]:
# if no driver is open, open one
def check_driver():
    try:
        driver.title
    
    # open if not open
        
    except WebDriverException:
        driver = webdriver.Chrome()

In [3]:
def replace_name_values(filename):
        # replace values with dashes for compatibility
    filename = filename.replace('%','_')
    filename = filename.replace('=','_')
    filename = filename.replace('?','_')
    filename = filename.replace('&','_')
    filename = filename.replace('20Season_','')
    filename = filename.replace('_20Season','')
    filename = filename.replace('SeasonType_','')
    filename = filename.replace('sort_gdate_dir_-1_','')
    filename = filename.replace('SeasonYear_','')
    return filename

In [32]:
def grab_team_data(url_list, file_folder):    
        i = 0
        for u in url_list:
                
                try:
                        driver.get(u)
                        src = driver.page_source
                        parser = BeautifulSoup(src, "lxml")
                        table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                except:
                        print(f'{u} Failed to load')
                        continue
                try: 
                        headers = table.findAll('th')
                        headerlist = [h.text.strip() for h in headers[0:]] 
                        row_names = table.findAll('a')                             # find rows
                        row_list = [b.text.strip() for b in row_names[0:]] 
                        rows = table.findAll('tr')[0:]
                except:
                        print(f'{u} Failed to load')
                        continue

                # get the data
                player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
                tot_cols = len(player_stats[1])                           #set the length to ignore hidden columns
                headerlist = headerlist[:tot_cols]   
                stats = pd.DataFrame(player_stats, columns = headerlist)

                # name file
                filename = file_folder + str(u[31:]).replace('/', '_') + '.csv'
                filename = replace_name_values(filename)

                # save to csv
                pd.DataFrame.to_csv(stats, filename)
                i += 1
                lu = len(url_list)
                print(f'{filename} Completed Successfully! {i} / {lu} Complete!')
        winsound.Beep(523, 500)

In [5]:
def team_data_filename_transformer(url):
    filename = str(url[31:]).replace('/', '_') + '.csv'
    filename = replace_name_values(filename)
    return filename

In [19]:
driver = webdriver.Chrome()

In [20]:
def grab_box_scores(url_list, file_folder):

        # Function to scrape box scores of individual games

        i = 0
        for u in url_list:
                driver.get(u)

                # click "all pages"
                xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                driver.find_element(by=By.XPATH, value=xpath_all).click()
                src = driver.page_source
                parser = BeautifulSoup(src, "lxml")

                # Find Table and scrape data
                table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                headers = table.findAll('th')
                headerlist = [h.text.strip() for h in headers[0:]] 
                row_names = table.findAll('a')                            
                row_list = [b.text.strip() for b in row_names[0:]] 
                rows = table.findAll('tr')[0:]
                player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
                tot_cols = len(player_stats[2])    
                headerlist = headerlist[:tot_cols]       
                stats = pd.DataFrame(player_stats, columns = headerlist)

                # File Name
                filename = file_folder + str(u[31:]).replace('/', '_') + '.csv'
                filename = replace_name_values(filename)
                filename = filename.replace('SeasonType_Regular_20Season','Reg_Season')

                # Save to CSV
                pd.DataFrame.to_csv(stats, filename)
                i += 1
                lu = len(url_list)
                print(f'{filename} Completed Successfully! {i} / {lu} Complete!')

In [21]:
def append_the_data(folder, data_prefix, filename_selector):

    # Appending data together via folder and/or file name
    
    path = folder
    p = os.listdir(path)
    pf = pd.DataFrame(p)

    # first filter
    pf_reg = pf.loc[pf[0].astype(str).str.contains(filename_selector)] 

    appended_data = pd.DataFrame()

    for file in pf_reg[0]:
        data = pd.read_csv(folder + '/' + file)
        # if Season is in the data, drop it
        if 'Season' in data.columns:
            data = data.drop(columns = ['Season'])

        # for some reason, this was the only way I could get the data to append correctly
        if '1996' in file:
            data['Season'] = 1996
        elif '1997' in file:
            data['Season'] = 1997
        elif '1998' in file:
            data['Season'] = 1998
        elif '1999' in file:
            data['Season'] = 1999
        elif '2000' in file:
            data['Season'] = 2000
        elif '2001' in file:
            data['Season'] = 2001
        elif '2002' in file:
            data['Season'] = 2002
        elif '2003' in file:
            data['Season'] = 2003
        elif '2004' in file:
            data['Season'] = 2004
        elif '2005' in file:
            data['Season'] = 2005
        elif '2006' in file:
            data['Season'] = 2006
        elif '2007' in file:
            data['Season'] = 2007
        elif '2008' in file:
            data['Season'] = 2008
        elif '2009' in file:
            data['Season'] = 2009
        elif '2010' in file:
            data['Season'] = 2010
        elif '2011' in file:
            data['Season'] = 2011
        elif '2012' in file:
            data['Season'] = 2012
        elif '2013' in file:
            data['Season'] = 2013
        elif '2014' in file:
            data['Season'] = 2014
        elif '2015' in file:
            data['Season'] = 2015
        elif '2016' in file:
            data['Season'] = 2016
        elif '2017' in file:
            data['Season'] = 2017
        elif '2018' in file:
            data['Season'] = 2018
        elif '2019' in file:
            data['Season'] = 2019
        elif '2020' in file:
            data['Season'] = 2020
        elif '2021' in file:
            data['Season'] = 2021
        elif '2022' in file:
            data['Season'] = 2022
        
        data['season_type'] = np.where('Reg' in file, 'Regular', 'Playoffs')
        # add prefix to columns
        data = data.add_prefix(data_prefix)
        data.columns = data.columns.str.lower()
        
        # save data to appended data
        appended_data = pd.DataFrame.append(appended_data, data)




    appended_data = pd.concat(appended_data)
    
    return appended_data

In [22]:
def quintuple_merge(df1, df2, df3, df4, df5, prefix1, prefix2, prefix3, prefix4, prefix5):

    # Merge 5 dataframes together

    
    merge_cols1 = [(prefix1 + 'team'), (prefix1 + 'matchup'), (prefix1 + 'gamedate') ,(prefix1 + 'season'), (prefix1 + 'season_type')]
    merge_cols2 = [(prefix2 + 'team'), (prefix2 + 'matchup'), (prefix2 + 'gamedate') ,(prefix2 + 'season'), (prefix2 + 'season_type')]
    merge_cols3 = [(prefix3 + 'team'), (prefix3 + 'matchup'), (prefix3 + 'gamedate') ,(prefix3 + 'season'), (prefix3 + 'season_type')]
    merge_cols4 = [(prefix4 + 'team'), (prefix4 + 'matchup'), (prefix4 + 'gamedate') ,(prefix4 + 'season'), (prefix4 + 'season_type')]
    merge_cols5 = [(prefix5 + 'team'), (prefix5 + 'matchup'), (prefix5 + 'gamedate') ,(prefix5 + 'season'), (prefix5 + 'season_type')]

    df = pd.merge(df1, df2, left_on= merge_cols1, right_on = merge_cols2, how='left')
    df = pd.merge(df,df3, left_on= merge_cols1, right_on = merge_cols3, how='left')
    df = pd.merge(df,df4, left_on= merge_cols1, right_on = merge_cols4, how='left')
    df = pd.merge(df,df5, left_on= merge_cols1, right_on = merge_cols5, how='left')

    return df

In [23]:
# Add folders if necessaryr
if os.path.isdir('data/team') is False:
    os.mkdir('data/team')

if os.path.isdir('data/team/team_general') is False:
    os.mkdir('data/team/team_general')

In [24]:
check_df = pd.read_csv('data/team/both_team_boxscores_ALL_with_GameDifferences.csv')
check_df.sort_values(by=['gamedate'], ascending = False, inplace=True)
check_df

,Unnamed: 0.1,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,...,t1_t2_ft_percent,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf,who_wins
60210,60210,60210,LAC,LAC @ TOR,12/31/2021,L,48.0,108.0,38.0,78.0,...,13.7,-15.0,-2.0,-17.0,8.0,-2.0,3.0,6.0,6.0,t2
60203,60203,60203,UTA,UTA vs. MIN,12/31/2021,W,48.0,120.0,37.0,78.0,...,21.8,-1.0,9.0,8.0,-5.0,-5.0,0.0,8.0,-10.0,t1
60205,60205,60205,IND,IND vs. CHI,12/31/2021,L,48.0,106.0,38.0,91.0,...,-10.8,9.0,4.0,13.0,1.0,-4.0,3.0,6.0,3.0,t2
60206,60206,60206,PHX,PHX @ BOS,12/31/2021,L,48.0,108.0,41.0,100.0,...,5.3,2.0,-16.0,-14.0,-8.0,4.0,-5.0,-8.0,3.0,t2
60207,60207,60207,MIN,MIN @ UTA,12/31/2021,L,48.0,108.0,40.0,93.0,...,-21.8,1.0,-9.0,-8.0,5.0,5.0,0.0,-8.0,10.0,t2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10080,10080,10080,MIN,MIN vs. HOU,01/01/2001,W,48.0,106.0,41.0,82.0,...,2.1,1.0,10.0,11.0,19.0,2.0,-1.0,-3.0,10.0,t1
10081,10081,10081,CHH,CHH @ POR,01/01/2001,L,48.0,67.0,26.0,80.0,...,-4.3,4.0,-10.0,-6.0,-4.0,-1.0,-3.0,1.0,8.0,t2
10082,10082,10082,HOU,HOU @ MIN,01/01/2001,L,48.0,96.0,31.0,77.0,...,-2.1,-1.0,-10.0,-11.0,-19.0,-2.0,1.0,3.0,-10.0,t2
3909,3909,3909,PHI,PHI @ VAN,01/01/1998,W,48.0,115.0,44.0,79.0,...,7.4,3.0,11.0,14.0,-3.0,3.0,-3.0,1.0,0.0,t1


## Team Season Data (General)

In [25]:
# make the url list of PREVIOUS SEASONS DATA
years = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', 
        '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', 
        '2003-04', '2002-03', '2001-02', '2000-01', '1999-00', '1998-99', '1997-98', '1996-97' ]
types = ['traditional', 'advanced', 'four-factors', 'scoring', 'opponent', 'defense']
season_types = ['Playoffs', 'Regular%20Season']

team_urlz = []

for year in years:
    for type in types:
        for s_types in season_types:
            url = 'https://www.nba.com/stats/teams/'+ type +'?SeasonType=' + s_types + '&Season=' + year
            team_urlz.append(str(url))

In [26]:
len(team_urlz)

312

#### Add 2022-23 Data

In [27]:
# get current date
today = datetime.date.today()

this_year_team_urls = []

for type in types:
    url = 'https://www.nba.com/stats/teams/'+ type +'?SeasonType=Regular%20Season&Season=2022-23'
    this_year_team_urls.append(str(url))

this_year_team_urls

['https://www.nba.com/stats/teams/traditional?SeasonType=Regular%20Season&Season=2022-23',
 'https://www.nba.com/stats/teams/advanced?SeasonType=Regular%20Season&Season=2022-23',
 'https://www.nba.com/stats/teams/four-factors?SeasonType=Regular%20Season&Season=2022-23',
 'https://www.nba.com/stats/teams/scoring?SeasonType=Regular%20Season&Season=2022-23',
 'https://www.nba.com/stats/teams/opponent?SeasonType=Regular%20Season&Season=2022-23',
 'https://www.nba.com/stats/teams/defense?SeasonType=Regular%20Season&Season=2022-23']

In [39]:
# check latest date in boxscores file
check_df = pd.read_csv('data/team/both_team_boxscores_ALL_with_GameDifferences.csv')
check_df['gamedate'] = pd.to_datetime(check_df['gamedate'])
check_df['gamedate'] = check_df['gamedate'].dt.date
check_df = check_df.sort_values(by=['gamedate'], ascending = False)
latest_date = check_df['gamedate'].iloc[0]
latest_date

datetime.date(2022, 6, 16)

In [40]:
check_df

,Unnamed: 0.1,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,...,t1_t2_ft_percent,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf,who_wins
65288,65288,65288,BOS,BOS vs. GSW,2022-06-16,L,48.0,90.0,34.0,80.0,...,-8.3,-4.0,1.0,-3.0,0.0,-5.0,1.0,6.0,-4.0,t2
65289,65289,65289,GSW,GSW @ BOS,2022-06-16,W,48.0,103.0,38.0,92.0,...,8.3,4.0,-1.0,3.0,0.0,5.0,-1.0,-6.0,4.0,t1
65290,65290,65290,BOS,BOS @ GSW,2022-06-13,L,48.0,94.0,31.0,75.0,...,-19.0,4.0,4.0,8.0,-5.0,-7.0,0.0,11.0,-12.0,t2
65291,65291,65291,GSW,GSW vs. BOS,2022-06-13,W,48.0,104.0,41.0,88.0,...,19.0,-4.0,-4.0,-8.0,5.0,7.0,0.0,-11.0,12.0,t1
65292,65292,65292,BOS,BOS vs. GSW,2022-06-10,L,48.0,97.0,34.0,85.0,...,-6.3,-5.0,-8.0,-13.0,2.0,-2.0,1.0,0.0,-4.0,t2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2355,2355,2355,LAC,LAC @ GSW,1996-11-01,W,48.0,97.0,41.0,86.0,...,-9.5,4.0,6.0,10.0,1.0,-1.0,-1.0,0.0,10.0,t1
2353,2353,2353,CHI,CHI @ BOS,1996-11-01,W,48.0,107.0,42.0,74.0,...,7.9,-6.0,7.0,1.0,8.0,-3.0,6.0,1.0,-10.0,t1
2352,2352,2352,DAL,DAL @ DEN,1996-11-01,W,48.0,92.0,35.0,88.0,...,-5.2,2.0,4.0,6.0,8.0,4.0,-3.0,1.0,9.0,t1
2350,2350,2350,MIA,MIA vs. ATL,1996-11-01,W,48.0,94.0,35.0,78.0,...,-10.7,3.0,-3.0,0.0,13.0,-2.0,-3.0,-5.0,5.0,t1


In [29]:
driver = webdriver.Chrome()

In [34]:
# if yesterday is later than latest date, then run the code
yesterday = today - datetime.timedelta(days=1)


if latest_date < yesterday:
    # get the data
    grab_team_data(this_year_team_urls, 'data/team/team_general/')
else:
    print('yesterday is not later than latest date, so do not run the code')

data/team/team_general/_traditional_Regular_Season_2022-23.csv Completed Successfully! 1 / 6 Complete!
data/team/team_general/_advanced_Regular_Season_2022-23.csv Completed Successfully! 2 / 6 Complete!
data/team/team_general/_four-factors_Regular_Season_2022-23.csv Completed Successfully! 3 / 6 Complete!
data/team/team_general/_scoring_Regular_Season_2022-23.csv Completed Successfully! 4 / 6 Complete!
data/team/team_general/_opponent_Regular_Season_2022-23.csv Completed Successfully! 5 / 6 Complete!
data/team/team_general/_defense_Regular_Season_2022-23.csv Completed Successfully! 6 / 6 Complete!


In [ ]:
def fix_box_url(url):
    name = team_data_filename_transformer(url)
    new_name = name.replace('Reg_Season_Season', 'Reg')
    new_name = new_name.replace('Playoffs_Season_Season', 'Playoffs')
    new_name = new_name.replace('SeasonYear', 'Yr')
    new_name = new_name.replace('SeasonType', '')
    return new_name

In [ ]:
def post_dl_fix(filename):
    new_name = filename.replace('Reg_Season_Season', 'Reg')
    new_name = new_name.replace('Playoffs_Season_Season', 'Playoffs')
    new_name = new_name.replace('SeasonYear', 'Yr')
    new_name = new_name.replace('SeasonType', '')
    return new_name

## Box Score Scraping

In [ ]:
types = ['boxscores-traditional', 'boxscores-advanced', 'boxscores-four-factors', 
        'boxscores-misc', 'boxscores-scoring' ]
season_types = ['Playoffs', 'Regular%20Season']

box_urls = []


for year in years:
    for type in types:
        for s_types in season_types:
            url = 'https://www.nba.com/stats/teams/'+ type +'?SeasonType=' + s_types + '&Season=' + year + "&SeasonYear=" + year
            box_urls.append(str(url))

print (len(box_urls))

260


In [ ]:
needed_boxes = pd.DataFrame(box_urls, columns=['url'])

needed_boxes['filenames'] = needed_boxes.apply(lambda row: fix_box_url(row['url']), axis=1)

needed_boxes = needed_boxes[~needed_boxes['filenames'].isin(team_box_files)]

needed_urls = needed_boxes['url'].tolist()

print(len(needed_urls))
needed_boxes

130


,url,filenames
1,https://www.nba.com/stats/teams/boxscores-trad...,_boxscores-traditional__Regular_20Season_Seaso...
3,https://www.nba.com/stats/teams/boxscores-adva...,_boxscores-advanced__Regular_20Season_Season_2...
5,https://www.nba.com/stats/teams/boxscores-four...,_boxscores-four-factors__Regular_20Season_Seas...
7,https://www.nba.com/stats/teams/boxscores-misc...,_boxscores-misc__Regular_20Season_Season_2021-...
9,https://www.nba.com/stats/teams/boxscores-scor...,_boxscores-scoring__Regular_20Season_Season_20...
...,...,...
251,https://www.nba.com/stats/teams/boxscores-trad...,_boxscores-traditional__Regular_20Season_Seaso...
253,https://www.nba.com/stats/teams/boxscores-adva...,_boxscores-advanced__Regular_20Season_Season_1...
255,https://www.nba.com/stats/teams/boxscores-four...,_boxscores-four-factors__Regular_20Season_Seas...
257,https://www.nba.com/stats/teams/boxscores-misc...,_boxscores-misc__Regular_20Season_Season_1996-...


In [ ]:
grab_box_scores(needed_urls, 'data/team/team_boxscores/')

data/team/team_boxscores/_boxscores-traditional_SeasonType_Playoffs_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 1 / 194 Complete!
data/team/team_boxscores/_boxscores-traditional_Reg_Season_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 2 / 194 Complete!
data/team/team_boxscores/_boxscores-advanced_Reg_Season_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 3 / 194 Complete!
data/team/team_boxscores/_boxscores-four-factors_Reg_Season_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 4 / 194 Complete!
data/team/team_boxscores/_boxscores-misc_Reg_Season_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 5 / 194 Complete!
data/team/team_boxscores/_boxscores-scoring_Reg_Season_Season_2021-22_SeasonYear_2021-22.csv Completed Successfully! 6 / 194 Complete!
data/team/team_boxscores/_boxscores-traditional_SeasonType_Playoffs_Season_2020-21_SeasonYear_2020-21.csv Completed Successfully! 7 / 194 Complete!
data/team/team_boxscor

In [ ]:
trad_files = os.listdir('data/team/team_boxscores/traditional')
trad_files

['_boxscores-traditional_Reg_1996-97_Yr_1996-97.csv',
 '_boxscores-traditional_Reg_1997-98_Yr_1997-98.csv',
 '_boxscores-traditional_Reg_1998-99_Yr_1998-99.csv',
 '_boxscores-traditional_Reg_1999-00_Yr_1999-00.csv',
 '_boxscores-traditional_Reg_2000-01_Yr_2000-01.csv',
 '_boxscores-traditional_Reg_2001-02_Yr_2001-02.csv',
 '_boxscores-traditional_Reg_2002-03_Yr_2002-03.csv',
 '_boxscores-traditional_Reg_2003-04_Yr_2003-04.csv',
 '_boxscores-traditional_Reg_2004-05_Yr_2004-05.csv',
 '_boxscores-traditional_Reg_2005-06_Yr_2005-06.csv',
 '_boxscores-traditional_Reg_2006-07_Yr_2006-07.csv',
 '_boxscores-traditional_Reg_2007-08_Yr_2007-08.csv',
 '_boxscores-traditional_Reg_2008-09_Yr_2008-09.csv',
 '_boxscores-traditional_Reg_2009-10_Yr_2009-10.csv',
 '_boxscores-traditional_Reg_2010-11_Yr_2010-11.csv',
 '_boxscores-traditional_Reg_2011-12_Yr_2011-12.csv',
 '_boxscores-traditional_Reg_2012-13_Yr_2012-13.csv',
 '_boxscores-traditional_Reg_2013-14_Yr_2013-14.csv',
 '_boxscores-traditional_Reg

In [ ]:
def get_seasontype(filename):
    if 'Playoffs' in filename:
        return 'Playoffs'
    else:
        return 'Regular'

In [ ]:
# append all trad_boxes to one file
trad_df = pd.DataFrame()

for file in trad_files:
    df = pd.read_csv('data/team/team_boxscores/traditional/' + file)
    # drop index
    df = df.drop(df.columns[0], axis=1)
    df.columns = df.columns.str.lower()
    df['season'] = file[-11:-7]
    df['season_type'] = get_seasontype(file)
    trad_df = trad_df.append(df)

trad_df = trad_df.dropna()
trad_df = trad_df.reset_index(drop=True)
trad_df

,team,match up,game date,w/l,min,pts,fgm,fga,fg%,3pm,...,dreb,reb,ast,tov,stl,blk,pf,+/-,season,season_type
0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,2.0,...,23.0,32.0,29.0,9.0,9.0,1.0,37.0,4.0,1996,Regular
1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,8.0,...,26.0,34.0,25.0,5.0,15.0,5.0,20.0,31.0,1996,Regular
2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,3.0,...,24.0,35.0,18.0,12.0,14.0,6.0,22.0,-4.0,1996,Regular
3,CHH,CHH @ MIL,04/20/1997,L,48.0,100.0,37.0,80.0,46.3,5.0,...,23.0,31.0,19.0,16.0,8.0,2.0,17.0,-20.0,1996,Regular
4,DET,DET @ IND,04/20/1997,W,53.0,124.0,42.0,80.0,52.5,10.0,...,29.0,37.0,19.0,12.0,6.0,4.0,25.0,4.0,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,PHI,PHI vs. TOR,04/16/2022,W,48.0,131.0,43.0,84.0,51.2,16.0,...,29.0,39.0,29.0,4.0,6.0,4.0,18.0,20.0,2021,Playoffs
65458,GSW,GSW vs. DEN,04/16/2022,W,48.0,123.0,43.0,82.0,52.4,16.0,...,31.0,41.0,33.0,13.0,5.0,8.0,22.0,16.0,2021,Playoffs
65459,MIN,MIN @ MEM,04/16/2022,W,48.0,130.0,45.0,90.0,50.0,16.0,...,35.0,46.0,32.0,19.0,5.0,7.0,32.0,13.0,2021,Playoffs
65460,UTA,UTA @ DAL,04/16/2022,W,48.0,99.0,36.0,83.0,43.4,7.0,...,40.0,53.0,15.0,14.0,3.0,5.0,25.0,6.0,2021,Playoffs


In [ ]:
trad_df.to_csv('data/team/team_boxscores/All_Traditional.csv')

In [ ]:
trad_df.season.unique()

array(['1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021'], dtype=object)

In [ ]:
adv_files = os.listdir('data/team/team_boxscores/advanced')

adv_df = pd.DataFrame()

for file in adv_files:
    df = pd.read_csv('data/team/team_boxscores/advanced/' + file)
    # drop index
    df = df.drop(df.columns[0], axis=1)
    df.columns = df.columns.str.lower()
    df['season'] = file[-11:-7]
    df['season_type'] = get_seasontype(file)
    adv_df = adv_df.append(df)

adv_df = adv_df.dropna()
adv_df = adv_df.reset_index(drop=True)
adv_df

,team,match up,game date,w/l,min,offrtg,defrtg,netrtg,ast%,ast/to,...,oreb%,dreb%,reb%,tov%,efg%,ts%,pace,pie,season,season_type
0,UTA,UTA @ SAC,04/20/1997,W,48.0,127.0,122.5,4.5,67.4,3.22,...,34.3,63.9,49.3,10.1,57.9,63.7,89.00,54.9,1996,Regular
1,TOR,TOR @ BOS,04/20/1997,W,48.0,126.3,94.0,32.3,51.0,5.00,...,19.1,63.6,40.7,5.1,57.6,60.7,99.50,67.3,1996,Regular
2,IND,IND vs. DET,04/20/1997,L,53.0,117.6,122.8,-5.1,61.4,1.93,...,49.2,80.5,62.0,13.7,48.0,53.2,91.92,48.3,1996,Regular
3,SAC,SAC vs. UTA,04/20/1997,L,48.0,120.2,127.0,-6.7,65.6,1.58,...,35.8,65.5,50.4,14.6,57.6,64.1,89.22,44.3,1996,Regular
4,CHH,CHH @ MIL,04/20/1997,L,48.0,102.0,125.0,-23.0,51.4,1.19,...,26.7,62.2,42.7,16.3,49.4,54.7,97.00,37.4,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,GSW,GSW vs. DEN,04/16/2022,W,48.0,130.9,113.8,17.0,76.7,2.54,...,34.1,66.7,52.2,13.8,62.2,64.9,94.00,57.8,2021,Playoffs
65458,UTA,UTA @ DAL,04/16/2022,W,48.0,108.8,103.3,5.5,41.7,1.07,...,39.6,82.4,61.6,15.4,47.6,53.2,90.50,53.7,2021,Playoffs
65459,MEM,MEM vs. MIN,04/16/2022,L,48.0,111.4,121.5,-10.1,64.1,2.08,...,25.0,67.4,44.9,11.4,49.4,55.8,106.00,44.6,2021,Playoffs
65460,DAL,DAL vs. UTA,04/16/2022,L,48.0,103.3,108.8,-5.5,58.6,2.43,...,17.6,60.4,38.4,7.8,44.1,51.1,90.50,46.3,2021,Playoffs


In [ ]:
adv_df.to_csv('data/team/team_boxscores/All_Advanced.csv')

In [ ]:
four_factors_files = os.listdir('data/team/team_boxscores/four-factors')

four_factors_df = pd.DataFrame()
for file in four_factors_files:
    df = pd.read_csv('data/team/team_boxscores/four-factors/' + file)
    # drop index
    df = df.drop(df.columns[0], axis=1)
    df.columns = df.columns.str.lower()
    df['season'] = file[-11:-7]
    df['season_type'] = get_seasontype(file)
    four_factors_df = four_factors_df.append(df)

four_factors_df = four_factors_df.dropna()
four_factors_df = four_factors_df.reset_index(drop=True)
four_factors_df

,team,match up,game date,w/l,min,efg%,ftarate,tov%,oreb%,oppefg%,oppfta rate,opptov%,opporeb%,season,season_type
0,SAS,SAS vs. HOU,04/20/1997,L,48.0,45.9,0.337,8.8,34.0,46.5,0.318,15.2,36.5,1996,Regular
1,TOR,TOR @ BOS,04/20/1997,W,48.0,57.6,0.272,5.1,19.1,48.7,0.382,24.0,36.4,1996,Regular
2,SAC,SAC vs. UTA,04/20/1997,L,48.0,57.6,0.953,14.6,35.8,57.9,0.382,10.1,34.5,1996,Regular
3,LAL,LAL @ POR,04/20/1997,L,48.0,48.2,0.293,12.9,27.7,55.1,0.500,20.7,34.2,1996,Regular
4,DET,DET @ IND,04/20/1997,W,53.0,58.8,0.488,11.9,19.5,48.0,0.290,13.7,49.2,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,GSW,GSW vs. DEN,04/16/2022,W,48.0,62.2,0.354,13.8,34.1,52.2,0.140,11.7,33.3,2021,Playoffs
65458,DEN,DEN @ GSW,04/16/2022,L,48.0,52.2,0.140,11.7,33.3,62.2,0.354,13.8,34.1,2021,Playoffs
65459,MIN,MIN @ MEM,04/16/2022,W,48.0,58.9,0.300,17.8,32.6,49.4,0.500,11.4,25.0,2021,Playoffs
65460,TOR,TOR @ PHI,04/16/2022,L,48.0,56.1,0.280,8.8,27.9,60.7,0.405,4.5,30.2,2021,Playoffs


In [ ]:
four_factors_df.to_csv('data/team/team_boxscores/All_Four_Factors.csv')

In [ ]:
misc_box_files = os.listdir('data/team/team_boxscores/misc')

misc_box_df = pd.DataFrame()
for file in misc_box_files:
    df = pd.read_csv('data/team/team_boxscores/misc/' + file)
    # drop index
    df = df.drop(df.columns[0], axis=1)
    df.columns = df.columns.str.lower()
    df['season'] = file[-11:-7]
    df['season_type'] = get_seasontype(file)
    misc_box_df = misc_box_df.append(df)

misc_box_df = misc_box_df.dropna()
misc_box_df = misc_box_df.reset_index(drop=True)
misc_box_df

,team,match up,game date,w/l,min,ptsoff to,2ndpts,fbps,pitp,opp ptsoff to,opp2nd pts,oppfbps,opppitp,season,season_type
0,LAL,LAL @ POR,04/20/1997,L,48.0,15.0,16.0,13.0,64.0,15.0,12.0,12.0,44.0,1996,Regular
1,TOR,TOR @ BOS,04/20/1997,W,48.0,27.0,15.0,30.0,68.0,4.0,14.0,12.0,34.0,1996,Regular
2,CLE,CLE vs. WAS,04/20/1997,L,48.0,13.0,10.0,8.0,40.0,10.0,26.0,5.0,46.0,1996,Regular
3,HOU,HOU @ SAS,04/20/1997,W,48.0,14.0,15.0,4.0,38.0,21.0,18.0,13.0,60.0,1996,Regular
4,MIL,MIL vs. CHH,04/20/1997,W,48.0,20.0,18.0,28.0,74.0,18.0,10.0,17.0,42.0,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,MEM,MEM vs. MIN,04/16/2022,L,48.0,25.0,14.0,18.0,60.0,11.0,19.0,9.0,50.0,2021,Playoffs
65458,DAL,DAL vs. UTA,04/16/2022,L,48.0,14.0,7.0,10.0,36.0,10.0,20.0,8.0,50.0,2021,Playoffs
65459,UTA,UTA @ DAL,04/16/2022,W,48.0,10.0,20.0,8.0,50.0,14.0,7.0,10.0,36.0,2021,Playoffs
65460,DEN,DEN @ GSW,04/16/2022,L,48.0,17.0,8.0,16.0,54.0,21.0,20.0,4.0,42.0,2021,Playoffs


In [ ]:
misc_box_df.to_csv('data/team/team_boxscores/All_Misc.csv')

In [ ]:
scoring_box_files = os.listdir('data/team/team_boxscores/scoring')

scoring_box_df = pd.DataFrame()
for file in scoring_box_files:
    df = pd.read_csv('data/team/team_boxscores/scoring/' + file)
    # drop index
    df = df.drop(df.columns[0], axis=1)
    df.columns = df.columns.str.lower()
    df['season'] = file[-11:-7]
    df['season_type'] = get_seasontype(file)
    scoring_box_df = scoring_box_df.append(df)

scoring_box_df = scoring_box_df.dropna()
scoring_box_df = scoring_box_df.reset_index(drop=True)
scoring_box_df

,team,match up,game date,w/l,min,%fga2pt,%fga3pt,%pts2pt,%pts2pt mr,%pts3pt,...,%ptsoff to,%ptspitp,2fgm%ast,2fgm%uast,3fgm%ast,3fgm%uast,fgm%ast,fgm%uast,season,season_type
0,TOR,TOR @ BOS,04/20/1997,W,48.0,76.1,23.9,65.6,11.2,19.2,...,21.6,54.4,46.3,53.7,75.0,25.0,51.0,49.0,1996,Regular
1,HOU,HOU @ SAS,04/20/1997,W,48.0,70.6,29.4,56.3,19.4,20.4,...,13.6,36.9,44.8,55.2,100.0,0.0,55.6,44.4,1996,Regular
2,UTA,UTA @ SAC,04/20/1997,W,48.0,90.8,9.2,72.6,14.2,5.3,...,17.7,58.4,65.9,34.1,100.0,0.0,67.4,32.6,1996,Regular
3,CHH,CHH @ MIL,04/20/1997,L,48.0,82.5,17.5,64.0,22.0,15.0,...,18.0,42.0,53.1,46.9,40.0,60.0,51.4,48.6,1996,Regular
4,POR,POR vs. LAL,04/20/1997,W,48.0,82.4,17.6,60.0,16.0,15.0,...,15.0,44.0,53.3,46.7,40.0,60.0,51.4,48.6,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,GSW,GSW vs. DEN,04/16/2022,W,48.0,57.3,42.7,43.9,9.8,39.0,...,17.1,34.1,81.5,18.5,68.8,31.3,76.7,23.3,2021,Playoffs
65458,PHI,PHI vs. TOR,04/16/2022,W,48.0,61.9,38.1,41.2,3.1,36.6,...,8.4,38.2,59.3,40.7,81.3,18.8,67.4,32.6,2021,Playoffs
65459,MEM,MEM vs. MIN,04/16/2022,L,48.0,68.6,31.4,54.7,3.4,17.9,...,21.4,51.3,56.3,43.8,100.0,0.0,64.1,35.9,2021,Playoffs
65460,DEN,DEN @ GSW,04/16/2022,L,48.0,62.4,37.6,59.8,9.3,30.8,...,15.9,50.5,50.0,50.0,90.9,9.1,60.5,39.5,2021,Playoffs


In [ ]:
scoring_box_df.to_csv('data/team/team_boxscores/All_Scoring.csv')

#### Fix dtypes

In [ ]:
four_factors_df.columns

Index(['team', 'match up', 'game date', 'w/l', 'min', 'efg%', 'ftarate',
       'tov%', 'oreb%', 'oppefg%', 'oppfta rate', 'opptov%', 'opporeb%',
       'season', 'season_type'],
      dtype='object')

In [ ]:
four_factors_df['efg%'] = four_factors_df['efg%'].astype(str)
four_factors_df['efg%'] = four_factors_df['efg%'].str.replace('%', '')
four_factors_df['efg%'] = four_factors_df['efg%'].astype(float)

four_factors_df['tov%'] = four_factors_df['tov%'].astype(str)
four_factors_df['tov%'] = four_factors_df['tov%'].str.replace('%', '')
four_factors_df['tov%'] = four_factors_df['tov%'].astype(float)

four_factors_df['opptov%'] = four_factors_df['opptov%'].astype(str)
four_factors_df['opptov%'] = four_factors_df['opptov%'].str.replace('%', '')
four_factors_df['opptov%'] = four_factors_df['opptov%'].astype(float)

four_factors_df['opporeb%'] = four_factors_df['opporeb%'].astype(str)
four_factors_df['opporeb%'] = four_factors_df['opporeb%'].str.replace('%', '')
four_factors_df['opporeb%'] = four_factors_df['opporeb%'].astype(float)

In [ ]:
trad_df.rename(columns = {'game\xa0date': 'gamedate'}, inplace = True)
trad_df.rename(columns = {'match\xa0up': 'matchup'}, inplace = True)
trad_df.rename(columns = {'game date': 'gamedate'}, inplace = True)
trad_df.rename(columns = {'match up': 'matchup'}, inplace = True)

adv_df.rename(columns = {'adv_game\xa0date': 'adv_gamedate'}, inplace = True)
adv_df.rename(columns = {'adv_match\xa0up': 'adv_matchup'}, inplace = True)
adv_df.rename(columns = {'adv_game date': 'adv_gamedate'}, inplace = True)
adv_df.rename(columns = {'adv_match up': 'adv_matchup'}, inplace = True)

four_factors_df.rename(columns = {'four_game\xa0date': 'four_gamedate'}, inplace = True)
four_factors_df.rename(columns = {'four_match\xa0up': 'four_matchup'}, inplace = True)
four_factors_df.rename(columns = {'four_game date': 'four_gamedate'}, inplace = True)
four_factors_df.rename(columns = {'four_match up': 'four_matchup'}, inplace = True)

misc_box_df.rename(columns = {'misc_game\xa0date': 'misc_gamedate'}, inplace = True)
misc_box_df.rename(columns = {'misc_match\xa0up': 'misc_matchup'}, inplace = True)
misc_box_df.rename(columns = {'misc_game date': 'misc_gamedate'}, inplace = True)
misc_box_df.rename(columns = {'misc_match up': 'misc_matchup'}, inplace = True)

scoring_box_df.rename(columns = {'scoring_game\xa0date': 'scoring_gamedate'}, inplace = True)
scoring_box_df.rename(columns = {'scoring_match\xa0up': 'scoring_matchup'}, inplace = True)
scoring_box_df.rename(columns = {'scoring_game date': 'scoring_gamedate'}, inplace = True)
scoring_box_df.rename(columns = {'scoring_match up': 'scoring_matchup'}, inplace = True)

### Save Advanced Boxes

In [ ]:
adv_df.to_csv('data/team/team_boxscores/ALL_advanced_boxes.csv')
trad_df.to_csv('data/team/team_boxscores/ALL_traditional_boxes.csv')
four_factors_df.to_csv('data/team/team_boxscores/ALL_four_factors_boxes.csv')
misc_box_df.to_csv('data/team/team_boxscores/ALL_misc_boxes.csv')
scoring_box_df.to_csv('data/team/team_boxscores/ALL_scoring_boxes.csv')

In [ ]:
trad_boxes = pd.read_csv('data/team/team_boxscores/ALL_traditional_boxes.csv')
adv_boxes = pd.read_csv('data/team/team_boxscores/ALL_advanced_boxes.csv')
four_factors_df = pd.read_csv('data/team/team_boxscores/ALL_four_factors_boxes.csv')
misc_boxes = pd.read_csv('data/team/team_boxscores/ALL_misc_boxes.csv')
scoring_boxes = pd.read_csv('data/team/team_boxscores/ALL_scoring_boxes.csv')

In [ ]:
# add prefixes to columns for each boxscore type (other than trad)
adv_boxes.columns = ['adv_' + str(col) for col in adv_boxes.columns]
four_factors_df.columns = ['four_' + str(col) for col in four_factors_df.columns]
misc_boxes.columns = ['misc_' + str(col) for col in misc_boxes.columns]
scoring_boxes.columns = ['scoring_' + str(col) for col in scoring_boxes.columns]

In [ ]:
# pre-merge check
print(f' traditional: {trad_boxes.shape}, advanced = {adv_boxes.shape}, four factors = {four_factors_df.shape}, misc = {misc_boxes.shape}, scoring = {scoring_boxes.shape}')

 traditional: (65462, 27), advanced = (65462, 22), four factors = (65462, 16), misc = (65462, 16), scoring = (65462, 23)


In [ ]:
adv_boxes.head(2)

,adv_Unnamed: 0,adv_team,adv_match up,adv_game date,adv_w/l,adv_min,adv_offrtg,adv_defrtg,adv_netrtg,adv_ast%,...,adv_oreb%,adv_dreb%,adv_reb%,adv_tov%,adv_efg%,adv_ts%,adv_pace,adv_pie,adv_season,adv_season_type
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,127.0,122.5,4.5,67.4,...,34.3,63.9,49.3,10.1,57.9,63.7,89.0,54.9,1996,Regular
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,126.3,94.0,32.3,51.0,...,19.1,63.6,40.7,5.1,57.6,60.7,99.5,67.3,1996,Regular


In [ ]:
adv_boxes.rename(columns = {'adv_game\xa0date': 'adv_gamedate'}, inplace = True)
adv_boxes.rename(columns = {'adv_match\xa0up': 'adv_matchup'}, inplace = True)
adv_boxes.rename(columns = {'adv_game date': 'adv_gamedate'}, inplace = True)
adv_boxes.rename(columns = {'adv_match up': 'adv_matchup'}, inplace = True)

In [ ]:
four_factors_df.rename(columns = {'four_game\xa0date': 'four_gamedate'}, inplace = True)
four_factors_df.rename(columns = {'four_match\xa0up': 'four_matchup'}, inplace = True)
four_factors_df.rename(columns = {'four_game date': 'four_gamedate'}, inplace = True)
four_factors_df.rename(columns = {'four_match up': 'four_matchup'}, inplace = True)

In [ ]:
misc_boxes.rename(columns = {'misc_game\xa0date': 'misc_gamedate'}, inplace = True)
misc_boxes.rename(columns = {'misc_match\xa0up': 'misc_matchup'}, inplace = True)
misc_boxes.rename(columns = {'misc_game date': 'misc_gamedate'}, inplace = True)
misc_boxes.rename(columns = {'misc_match up': 'misc_matchup'}, inplace = True)

scoring_boxes.rename(columns = {'scoring_game\xa0date': 'scoring_gamedate'}, inplace = True)
scoring_boxes.rename(columns = {'scoring_match\xa0up': 'scoring_matchup'}, inplace = True)
scoring_boxes.rename(columns = {'scoring_game date': 'scoring_gamedate'}, inplace = True)
scoring_boxes.rename(columns = {'scoring_match up': 'scoring_matchup'}, inplace = True)


In [ ]:
all_boxes = quintuple_merge(trad_boxes, adv_boxes, four_factors_df, misc_boxes, scoring_boxes, '', 'adv_', 'four_', 'misc_', 'scoring_')
all_boxes

,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,fg%,...,scoring_%ptsoff to,scoring_%ptspitp,scoring_2fgm%ast,scoring_2fgm%uast,scoring_3fgm%ast,scoring_3fgm%uast,scoring_fgm%ast,scoring_fgm%uast,scoring_season,scoring_season_type
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,17.7,58.4,65.9,34.1,100.0,0.0,67.4,32.6,1996,Regular
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,21.6,54.4,46.3,53.7,75.0,25.0,51.0,49.0,1996,Regular
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,15.6,66.7,45.7,54.3,66.7,33.3,47.4,52.6,1996,Regular
3,3,CHH,CHH @ MIL,04/20/1997,L,48.0,100.0,37.0,80.0,46.3,...,18.0,42.0,53.1,46.9,40.0,60.0,51.4,48.6,1996,Regular
4,4,DET,DET @ IND,04/20/1997,W,53.0,124.0,42.0,80.0,52.5,...,14.5,29.0,34.4,65.6,80.0,20.0,45.2,54.8,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,65457,PHI,PHI vs. TOR,04/16/2022,W,48.0,131.0,43.0,84.0,51.2,...,8.4,38.2,59.3,40.7,81.3,18.8,67.4,32.6,2021,Playoffs
65458,65458,GSW,GSW vs. DEN,04/16/2022,W,48.0,123.0,43.0,82.0,52.4,...,17.1,34.1,81.5,18.5,68.8,31.3,76.7,23.3,2021,Playoffs
65459,65459,MIN,MIN @ MEM,04/16/2022,W,48.0,130.0,45.0,90.0,50.0,...,8.5,38.5,62.1,37.9,87.5,12.5,71.1,28.9,2021,Playoffs
65460,65460,UTA,UTA @ DAL,04/16/2022,W,48.0,99.0,36.0,83.0,43.4,...,10.1,50.5,34.5,65.5,71.4,28.6,41.7,58.3,2021,Playoffs


In [ ]:
#find all unnamed columns
unnamed_cols = [col for col in all_boxes.columns if 'Unnamed' in col]
unnamed_cols

['Unnamed: 0',
 'adv_Unnamed: 0',
 'four_Unnamed: 0',
 'misc_Unnamed: 0',
 'scoring_Unnamed: 0']

In [ ]:
all_boxes = all_boxes.drop(unnamed_cols, axis = 1)

In [ ]:
all_boxes.to_csv('data/team/team_boxscores/ALL_BOXES.csv')

#### Load all_boxes

In [ ]:
all_boxes = pd.read_csv('data/team/team_boxscores/ALL_BOXES.csv')

## Fixing up all_boxes

We have 68000 rows, with 2 rows per game, meaning approximately 34,000 games. 

I need to match games up now

In [ ]:
all_boxes_copy = all_boxes.copy()
all_boxes_copy = all_boxes_copy.add_prefix('tm2__')

In [ ]:
print(f' all_boxes shape: {all_boxes.shape}, all_boxes_copy shape: {all_boxes_copy.shape}')

 all_boxes shape: (65462, 100), all_boxes_copy shape: (65462, 100)


In [ ]:
# add game id and second team to all_boxes
all_boxes['matchup'] = all_boxes['matchup'].astype(str)
all_boxes['team_2'] = all_boxes['matchup'].str[-3:]
all_boxes.head(3)

,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,fg%,...,scoring_%ptspitp,scoring_2fgm%ast,scoring_2fgm%uast,scoring_3fgm%ast,scoring_3fgm%uast,scoring_fgm%ast,scoring_fgm%uast,scoring_season,scoring_season_type,team_2
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,58.4,65.9,34.1,100.0,0.0,67.4,32.6,1996,Regular,SAC
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,54.4,46.3,53.7,75.0,25.0,51.0,49.0,1996,Regular,BOS
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,66.7,45.7,54.3,66.7,33.3,47.4,52.6,1996,Regular,POR


In [ ]:
all_boxes['game_id'] = np.where(all_boxes['matchup'].str.contains('vs'), 
                                all_boxes['matchup'].str[-3:] + ' @ ' + 
                                all_boxes['matchup'].str[0:3] + '_' +
                                all_boxes['gamedate'].astype(str), 
                                all_boxes['matchup'].astype(str) +  '_' + all_boxes['gamedate'].astype(str))

In [ ]:
# create Game_id 
all_boxes_copy['game_id'] = np.where(all_boxes_copy['tm2__matchup'].str.contains('vs'), 
                                all_boxes_copy['tm2__matchup'].str[-3:] + ' @ ' + 
                                all_boxes_copy['tm2__matchup'].str[0:3] + '_' +
                                all_boxes_copy['tm2__gamedate'].astype(str), 
                                all_boxes_copy['tm2__matchup'].astype(str) +  '_' + all_boxes_copy['tm2__gamedate'].astype(str))
all_boxes_copy.head(3)

,tm2__Unnamed: 0,tm2__team,tm2__matchup,tm2__gamedate,tm2__w/l,tm2__min,tm2__pts,tm2__fgm,tm2__fga,tm2__fg%,...,tm2__scoring_%ptspitp,tm2__scoring_2fgm%ast,tm2__scoring_2fgm%uast,tm2__scoring_3fgm%ast,tm2__scoring_3fgm%uast,tm2__scoring_fgm%ast,tm2__scoring_fgm%uast,tm2__scoring_season,tm2__scoring_season_type,game_id
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,58.4,65.9,34.1,100.0,0.0,67.4,32.6,1996,Regular,UTA @ SAC_04/20/1997
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,54.4,46.3,53.7,75.0,25.0,51.0,49.0,1996,Regular,TOR @ BOS_04/20/1997
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,66.7,45.7,54.3,66.7,33.3,47.4,52.6,1996,Regular,LAL @ POR_04/20/1997


In [ ]:
check1 = all_boxes[['game_id', 'team_2']]
check2 = all_boxes_copy[['game_id', 'tm2__team']]

game_id    object
team_2     object
dtype: object

In [ ]:
all_boxes.game_id = all_boxes.game_id.astype(str)
all_boxes_copy.game_id = all_boxes_copy.game_id.astype(str)
all_boxes_copy.tm2__=team = all_boxes_copy.tm2__team.astype(str)
all_boxes.team_2 = all_boxes.team_2.astype(str)

In [ ]:
# merge the dfs 
boxes_both_teams = pd.merge(all_boxes, all_boxes_copy, 
                    left_on = ['game_id', 'team_2'], 
                    right_on = ['game_id', 'tm2__team'],
                    how = 'left')
boxes_both_teams

,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,fg%,...,tm2__scoring_%ptsoff to,tm2__scoring_%ptspitp,tm2__scoring_2fgm%ast,tm2__scoring_2fgm%uast,tm2__scoring_3fgm%ast,tm2__scoring_3fgm%uast,tm2__scoring_fgm%ast,tm2__scoring_fgm%uast,tm2__scoring_season,tm2__scoring_season_type
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,11.0,23.9,57.7,42.3,100.0,0.0,65.6,34.4,1996,Regular
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,4.3,36.2,52.0,48.0,100.0,0.0,63.6,36.4,1996,Regular
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,15.0,44.0,53.3,46.7,40.0,60.0,51.4,48.6,1996,Regular
3,3,CHH,CHH @ MIL,04/20/1997,L,48.0,100.0,37.0,80.0,46.3,...,16.7,61.7,57.1,42.9,66.7,33.3,57.7,42.3,1996,Regular
4,4,DET,DET @ IND,04/20/1997,W,53.0,124.0,42.0,80.0,52.5,...,15.8,45.0,55.6,44.4,87.5,12.5,61.4,38.6,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,65457,PHI,PHI vs. TOR,04/16/2022,W,48.0,131.0,43.0,84.0,51.2,...,3.6,43.2,53.6,46.4,100.0,0.0,67.5,32.5,2021,Playoffs
65458,65458,GSW,GSW vs. DEN,04/16/2022,W,48.0,123.0,43.0,82.0,52.4,...,15.9,50.5,50.0,50.0,90.9,9.1,60.5,39.5,2021,Playoffs
65459,65459,MIN,MIN @ MEM,04/16/2022,W,48.0,130.0,45.0,90.0,50.0,...,21.4,51.3,56.3,43.8,100.0,0.0,64.1,35.9,2021,Playoffs
65460,65460,UTA,UTA @ DAL,04/16/2022,W,48.0,99.0,36.0,83.0,43.4,...,15.1,38.7,40.0,60.0,100.0,0.0,58.6,41.4,2021,Playoffs


In [ ]:
boxes_both_teams = boxes_both_teams.drop_duplicates()
boxes_both_teams2 = boxes_both_teams.dropna(subset = ['tm2__scoring_season'])

In [ ]:
boxes_both_teams2

,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,fg%,...,tm2__scoring_%ptsoff to,tm2__scoring_%ptspitp,tm2__scoring_2fgm%ast,tm2__scoring_2fgm%uast,tm2__scoring_3fgm%ast,tm2__scoring_3fgm%uast,tm2__scoring_fgm%ast,tm2__scoring_fgm%uast,tm2__scoring_season,tm2__scoring_season_type
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,11.0,23.9,57.7,42.3,100.0,0.0,65.6,34.4,1996,Regular
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,4.3,36.2,52.0,48.0,100.0,0.0,63.6,36.4,1996,Regular
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,15.0,44.0,53.3,46.7,40.0,60.0,51.4,48.6,1996,Regular
3,3,CHH,CHH @ MIL,04/20/1997,L,48.0,100.0,37.0,80.0,46.3,...,16.7,61.7,57.1,42.9,66.7,33.3,57.7,42.3,1996,Regular
4,4,DET,DET @ IND,04/20/1997,W,53.0,124.0,42.0,80.0,52.5,...,15.8,45.0,55.6,44.4,87.5,12.5,61.4,38.6,1996,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,65457,PHI,PHI vs. TOR,04/16/2022,W,48.0,131.0,43.0,84.0,51.2,...,3.6,43.2,53.6,46.4,100.0,0.0,67.5,32.5,2021,Playoffs
65458,65458,GSW,GSW vs. DEN,04/16/2022,W,48.0,123.0,43.0,82.0,52.4,...,15.9,50.5,50.0,50.0,90.9,9.1,60.5,39.5,2021,Playoffs
65459,65459,MIN,MIN @ MEM,04/16/2022,W,48.0,130.0,45.0,90.0,50.0,...,21.4,51.3,56.3,43.8,100.0,0.0,64.1,35.9,2021,Playoffs
65460,65460,UTA,UTA @ DAL,04/16/2022,W,48.0,99.0,36.0,83.0,43.4,...,15.1,38.7,40.0,60.0,100.0,0.0,58.6,41.4,2021,Playoffs


In [ ]:
rbbt = boxes_both_teams2

In [ ]:
rbbt.to_csv('data/team/both_team_boxscores_ALL.csv')

### Add Features

In [ ]:
rbbt = rbbt.assign(t1_t2_pts = rbbt['pts'] - rbbt['tm2__pts'])
rbbt = rbbt.assign(t1_t2_fgm = rbbt['fgm'] - rbbt['tm2__fgm'])
rbbt = rbbt.assign(t1_t2_fga = rbbt['fga'] - rbbt['tm2__fga'])
rbbt = rbbt.assign(t1_t2_fg_percent = rbbt['fg%'] - rbbt['tm2__fg%'])
rbbt = rbbt.assign(t1_t2_3pm = rbbt['3pm'] - rbbt['tm2__3pm'])
rbbt = rbbt.assign(t1_t2_3pa = rbbt['3pa'] - rbbt['tm2__3pa'])
rbbt = rbbt.assign(t1_t2_3p_percent = rbbt['3p%'] - rbbt['tm2__3p%'])
rbbt = rbbt.assign(t1_t2_ftm = rbbt['ftm'] - rbbt['tm2__ftm'])
rbbt = rbbt.assign(t1_t2_fta = rbbt['fta'] - rbbt['tm2__fta'])
rbbt = rbbt.assign(t1_t2_ft_percent = rbbt['ft%'] - rbbt['tm2__ft%'])
rbbt = rbbt.assign(t1_t2_oreb = rbbt['oreb'] - rbbt['tm2__oreb'])
rbbt = rbbt.assign(t1_t2_dreb = rbbt['dreb'] - rbbt['tm2__dreb'])
rbbt = rbbt.assign(t1_t2_reb = rbbt['reb'] - rbbt['tm2__reb'])
rbbt = rbbt.assign(t1_t2_ast = rbbt['ast'] - rbbt['tm2__ast'])
rbbt = rbbt.assign(t1_t2_stl = rbbt['stl'] - rbbt['tm2__stl'])
rbbt = rbbt.assign(t1_t2_blk = rbbt['blk'] - rbbt['tm2__blk'])
rbbt = rbbt.assign(t1_t2_tov = rbbt['tov'] - rbbt['tm2__tov'])
rbbt = rbbt.assign(t1_t2_pf = rbbt['pf'] - rbbt['tm2__pf'])

In [ ]:
rbbt

,Unnamed: 0,team,matchup,gamedate,w/l,min,pts,fgm,fga,fg%,...,t1_t2_fta,t1_t2_ft_percent,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf
0,0,UTA,UTA @ SAC,04/20/1997,W,48.0,113.0,43.0,76.0,56.6,...,-28.0,17.8,-1.0,2.0,1.0,8.0,6.0,-3.0,-4.0,14.0
1,1,TOR,TOR @ BOS,04/20/1997,W,48.0,125.0,49.0,92.0,53.3,...,-4.0,7.0,-2.0,-9.0,-11.0,4.0,12.0,5.0,-19.0,-2.0
2,2,LAL,LAL @ POR,04/20/1997,L,48.0,96.0,38.0,82.0,46.3,...,-10.0,-2.7,0.0,-5.0,-5.0,0.0,7.0,4.0,-7.0,2.0
3,3,CHH,CHH @ MIL,04/20/1997,L,48.0,100.0,37.0,80.0,46.3,...,6.0,15.8,-4.0,-9.0,-13.0,-11.0,-2.0,-2.0,1.0,-3.0
4,4,DET,DET @ IND,04/20/1997,W,53.0,124.0,42.0,80.0,52.5,...,10.0,-5.9,-16.0,-2.0,-18.0,-8.0,-1.0,1.0,-2.0,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65457,65457,PHI,PHI vs. TOR,04/16/2022,W,48.0,131.0,43.0,84.0,51.2,...,11.0,2.7,3.0,1.0,4.0,2.0,5.0,-4.0,-4.0,-8.0
65458,65458,GSW,GSW vs. DEN,04/16/2022,W,48.0,123.0,43.0,82.0,52.4,...,16.0,-4.5,1.0,5.0,6.0,7.0,-3.0,5.0,2.0,0.0
65459,65459,MIN,MIN @ MEM,04/16/2022,W,48.0,130.0,45.0,90.0,50.0,...,-16.0,14.5,3.0,8.0,11.0,7.0,-2.0,-1.0,7.0,8.0
65460,65460,UTA,UTA @ DAL,04/16/2022,W,48.0,99.0,36.0,83.0,43.4,...,-11.0,10.5,6.0,13.0,19.0,-2.0,-5.0,1.0,7.0,3.0


In [ ]:
new_cols = ['t1_t2_pts', 't1_t2_fgm', 't1_t2_fga', 't1_t2_fg_percent', 't1_t2_3pm', 't1_t2_3pa', 't1_t2_3p_percent', 't1_t2_ftm', 't1_t2_fta', 't1_t2_ft_percent', 't1_t2_oreb', 't1_t2_dreb', 't1_t2_reb', 't1_t2_ast', 't1_t2_stl', 't1_t2_blk', 't1_t2_tov', 't1_t2_pf']

In [ ]:
rbbt['who_wins'] = np.where(rbbt['w/l'] == 'W', 't1', 't2')

In [ ]:
rbbt.to_csv('data/team/both_team_boxscores_ALL_with_GameDifferences.csv')